## Import Package

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Add, BatchNormalization, Activation, Flatten, LeakyReLU, Dropout
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.utils import to_categorical
import pandas as pd
import cv2
import os

## Load and preprocess training and testing dataset

In [ ]:
def printmd(string):
    # Print with Markdowns    
    display(Markdown(string))
    
def load_images_from_folder(folder,only_path = False, label = ""):
# Load the paths to the images in a directory
# or load the images
    if only_path == False:
        images = []
        for filename in os.listdir(folder):
            img = plt.imread(os.path.join(folder,filename))
            if img is not None:
                images.append(img)
        return images
    else:
        path = []
        for filename in os.listdir(folder):
            img_path = os.path.join(folder,filename)
            if img_path is not None:
                path.append([label,img_path])
        return path

In [ ]:
images_train = []
images_test = []
# dirp = "/kaggle/input/fruits/fruits-360_dataset/fruits-360/Training"

dir_train = "./fruits/train/"
dir_test = "./fruits/test/"

skip_list = []

for folder in os.listdir(dir_train):
    if folder in skip_list: continue
    
    images_train += load_images_from_folder(dir_train+folder, True, label = folder)

for folder in os.listdir(dir_test):
    if folder in skip_list: continue
    
    images_test += load_images_from_folder(dir_test+folder, True, label = folder)


df_train = pd.DataFrame(images_train, columns = ["food", "path"])
df_test = pd.DataFrame(images_test, columns = ["food", "path"])

# Shuffle the dataset
from sklearn.utils import shuffle
df_train = shuffle(df_train, random_state = 0)
df_train = df_train.reset_index(drop=True)
df_test = shuffle(df_test, random_state = 0)
df_test = df_test.reset_index(drop=True)

fruit_names = []
for folder in os.listdir(dir_train):
    fruit_names.append(folder)

mapper_fruit_names = dict(zip(fruit_names, [t for t in range(len(fruit_names))]))
df_train["label"] = df_train["food"].map(mapper_fruit_names)
df_test["label"] = df_test["food"].map(mapper_fruit_names)

df_train["food"].value_counts()
df_test["food"].value_counts()

In [ ]:
img_size = 144
RGBSETTINGS = 3

In [ ]:
def load_img(df):
    img_paths = df["path"].values       # path
    img_labels = df["label"].values     # label
    img_food = df["food"].values      # food
    X = []                              
    y = []

    for i, path in enumerate(img_paths):
        
        img = plt.imread(path)
        if RGBSETTINGS == 1:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img = cv2.resize(img, (img_size, img_size))
        label = img_labels[i]
        
        X.append(img)
        y.append(label)
        
    return np.array(X), np.array(y)

## Load and process training and testing dataset

In [ ]:
# Dataset preprocessing #1
train_images_database, train_labels_database = load_img(df_train)
test_images_database, test_labels_database = load_img(df_test)

num_classes = 10
input_shape = (img_size, img_size, RGBSETTINGS)
# Transfer to nparray
train_images = train_images_database.astype('float32')
train_labels = to_categorical(train_labels_database, num_classes, dtype = 'float32')
test_images = test_images_database.astype('float32')
test_labels = to_categorical(test_labels_database, num_classes, dtype = 'float32')

train_images = (train_images - 128.0) / 128.0

test_images = (test_images - 128.0) / 128.0

## Model define and create

In [ ]:
# Define the residual block
def residual_block(inputs, filters):
    x = Conv2D(filters, kernel_size=(3, 3), padding='same')(inputs)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = Conv2D(filters, kernel_size=(3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    
    # Adjust dimensions if needed
    # if inputs.shape[-1] != filters:
    inputs = Conv2D(filters, kernel_size=(1, 1), padding='same')(inputs)
    
    x = Add()([x, inputs])
    # x = Activation('relu')(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = Dropout(0.45)(x)
    return x

# Build the ResNet model
def build_resnet(input_shape, num_classes):
    inputs = keras.Input(shape=input_shape)
    x = Conv2D(8, kernel_size=(3, 3), padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)

    x = residual_block(x, filters=16)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    
    x = residual_block(x, filters=16)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    
    x = residual_block(x, filters=32)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    
    x = residual_block(x, filters=32)
    x = MaxPooling2D(pool_size=(2, 2))(x)


    x = Flatten()(x)
    x = Dense(64, activation='relu')(x)
    x = Dense(num_classes, activation='sigmoid')(x)

    model = Model(inputs=inputs, outputs=x)
    return model

# Define the input shape and number of classes
input_shape = (img_size, img_size, RGBSETTINGS)  # Modify according to your input image size
num_classes = 10  # Modify according to your number of classes

# Build and compile the model
model = build_resnet(input_shape, num_classes)

# Print the model summary
model.summary()

## Model training

In [ ]:
# Training model

#optimizer = 'RMSprop'
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# Set training
model.fit(train_images, train_labels,
          validation_split = 0.2,
          batch_size = 200,
          verbose = 1,
          epochs = 50
          )

In [ ]:
# Model Evaluation
score = model.evaluate(test_images, test_labels, verbose = 0)

print('test loss', score[0])
print('accuracy', score[1])


# Save weights of this model  
model.save_weights('my_model.h5')

# load weights to this TensorFlow model  
model.load_weights('my_model.h5')

In [ ]:
# Save model and weights of this model
model.save('model_save')

In [ ]:
# lOAD model and weights of this model
model_2 = keras.models.load_model('model_save')

# Model Evaluation
score = model_2.evaluate(test_images, test_labels, verbose = 0)
print('test loss', score[0])
print('accuracy', score[1])

In [ ]:
model = model_2

---------------------------------------------------------------------------------------------------------------------

## Reload and preprocess images in TFLM

In [ ]:
# Import training and testing from dataset_buffer

test_images = (test_images - 128.0) / 128.0

## Convert model into TFLM format

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8

In [ ]:
test_images = tf.cast(test_images, tf.float32)
tf_lite_ds = tf.data.Dataset.from_tensor_slices((test_images)).batch(1) #construct a dataset 

def representative_data_gen():
    for input_value in tf_lite_ds.take(100):
        yield [input_value]
    
converter.representative_dataset = representative_data_gen

In [ ]:
import pathlib

converted_model = converter.convert()

generated_dir = pathlib.Path("generated/")
generated_dir.mkdir(exist_ok=True, parents=True)
converted_model_file = generated_dir/"emnist_model_int8.tflite"
converted_model_file.write_bytes(converted_model)

In order to integrate converted model into TFLM application we have to save it as a C array. One way to do that is to use **xxd** utility available on Linux or in Cygwin/MinGW terminals on Windows. Open terminal and run following commands:

```
cd generated/
xxd -i emnist_model_int8.tflite > model.h
```

The model is ready to be integrated into TFLM application.

## Evaluate TensorFlow Lite INT-8 Model

Full test set contains 14800 samples. Evaluating int8 model on it might take more than 10 minutes. 
If you want to get estimation faster, please, limit number of samples to be evaluated by reducing **max_samples** value

In [ ]:
max_samples = int(len(test_images) * 1.0)

print(max_samples)

In [ ]:
import pathlib

generated_dir = pathlib.Path("generated/")
generated_dir.mkdir(exist_ok=True, parents=True)
converted_model_file = generated_dir/"emnist_model_int8.tflite"

interpreter = tf.lite.Interpreter(model_path=str(converted_model_file))
interpreter.allocate_tensors()

# A helper function to evaluate the TF Lite model using "test" dataset.
def evaluate_model(interpreter):
    input_index = interpreter.get_input_details()[0]["index"]
    output_index = interpreter.get_output_details()[0]["index"]
    scale, zero_point = interpreter.get_output_details()[0]['quantization']
    # print(scale, zero_point)
    prediction_values = []
    
    for tflm_test_image in test_images[:max_samples]:
        # Pre-processing: add batch dimension, quantize and convert inputs to int8 to match with
        # the model's input data format.
        tflm_test_image = np.expand_dims(tflm_test_image, axis=0) #.astype(np.float32)
        tflm_test_image = np.int8(tflm_test_image / scale + zero_point)
        interpreter.set_tensor(input_index, tflm_test_image)

        interpreter.invoke()

        # Find the letter with highest probability
        output = interpreter.tensor(output_index)
        result = np.argmax(output()[0])
        prediction_values.append(result)
    
    accurate_count = 0
    for index in range(len(prediction_values)):
        print(prediction_values[index], ans_test_labels[index])
        if prediction_values[index] == ans_test_labels[index]:
            accurate_count += 1
    accuracy = accurate_count * 1.0 / len(prediction_values)

    return accuracy * 100

In [ ]:
# Import training and testing from dataset_buffer
tflm_test_images = test_images_database
ans_test_labels = test_labels_database

def thinning(image):
    return np.where(image < 210.0, 0, 255)

tflm_test_images = (tflm_test_images - 128.0) / 128.0

Please, keep in mind that full test dataset evaluation on int8 model may take several minutes. 

In [ ]:
print(str(evaluate_model(interpreter)) + "%")

Evaluate the accuracy without normalizing data

In [ ]:
# Import training and testing from dataset_buffer
tflm_test_images = test_images_database
ans_test_labels = test_labels_database

tflm_test_images = (tflm_test_images - 128.0) / 128.0

In [ ]:
print(str(evaluate_model(interpreter)) + "%")

-----

## Create a test set for target application

In [ ]:
import random

# Import training and testing from dataset_buffer
test_images = test_images_database
test_labels = test_labels_database

test_images = test_images.reshape([test_images.shape[0], img_size, img_size, RGBSETTINGS])

num_of_samples = 25
random_test_images = random.sample(range(1, test_images.shape[0]), num_of_samples)

fig=plt.figure(figsize=(10, 10))
rows = 5
cols = 5

for index in range(0, num_of_samples):
    img = test_images[random_test_images[index]]
    fig.add_subplot(rows, cols, (index + 1))
    plt.imshow(img)

plt.show()


In [ ]:
samples_file = open("generated/test_samples.cpp", "w")

samples_file.write("#include \"test_samples.h\"\n\n")
samples_file.write("const int kNumSamples = " + str(num_of_samples) + ";\n\n")

samples = "" 
samples_array = "const TestSample test_samples[kNumSamples] = {"

for sample_idx, img_idx in enumerate(random_test_images, 1):
    img_arr = list(np.ndarray.flatten(test_images[img_idx]))
    var_name = "sample" + str(sample_idx)
    samples += "TestSample " + var_name + " = {\n" #+ "[IMAGE_SIZE] = { "
    samples += "\t.label = " + str(test_labels[img_idx]) + ",\n" 
    samples += "\t.image = {\n"
    wrapped_arr = [img_arr[i:i + 20] for i in range(0, len(img_arr), 20)]
    for sub_arr in wrapped_arr:
        samples += "\t\t" + str(sub_arr)
    samples += "\t}\n};\n\n"    
    samples_array += var_name + ", "
    
samples = samples.replace("[", "")
samples = samples.replace("]", ",\n")
samples_array += "};\n"

samples_file.write(samples);
samples_file.write(samples_array);
samples_file.close()

## Done

You have converted a Tensorflow model into TFLM format and generated a test set for the application. Now you can copy generated files into target application of this tutorial and try it out:

In order to integrate converted model into TFLM application we have to save it as a C array. One way to do that is to use **xxd** utility available on Linux or in Cygwin/MinGW terminals on Windows. Open terminal and run following commands:

```
cd generated/
xxd -i emnist_model_int8.tflite > model.h
```

The model is ready to be integrated into TFLM application.

* copy *generated/model.h* to *../inc* and *generated/test_samples.cpp* to *../src*
* You can start to integrate your WE-I project

In [ ]:
!xxd -i generated/emnist_model_int8.tflite > model.h